# Ejemplo real de uso con información de la EOD 2012

La mayoria de los ejemplos fueron obtenidos de una presentación de **Eduardo Graells-Garrido**

**Es recomendable trabajar este archivo en Colab de Google**

En este ejemplo se analiza la encuesta Origen Destino de Santiago, realizada el año 2012. 

La Encuesta Origen-Destino de Santiago, efectuada por última vez el año 2012, es el instrumento principal que utilizan las autoridades para tomar decisiones respecto a transporte en la ciudad. Consistió en entrevistar a los residentes de más de 18000 hogares haciéndoles la siguiente pregunta:

> ¿Qué viajes hiciste ayer?

Las personas encuestadas responden a través de un diario de viaje. En este diario incluyen todos los datos pertinentes de sus viajes: a qué hora lo iniciaron, a qué hora terminaron, los puntos de origen y destino (coordenadas), el propósito del viaje, el/los modo(s) de viaje utilizados, etc. También incluye información socio-demográfica de cada persona que responde.

La encuesta es representativa a nivel comunal. Esto quiere decir que podemos sacar conclusiones sobre como se moviliza la población de Providencia, pero no de un barrio específico de la comuna. Puede ser que exista información de ese barrio específico, pero no podemos sacar conclusiones extrapolables al barrio completo.

## Datasets

En este ejemplo trabajaremos con la base de datos de la EOD disponible en la página de datos del Gobierno disponible en el siguiente link.

  * [Encuesta Origen-Destino Santiago 2012](http://datos.gob.cl/dataset/31616)

Con el comando **git clone** descargaremos a la memoria de este notebook la información requerida para trabajar.

In [ ]:
!git clone https://github.com/carnby/carto-en-python.git 


## Previo

Primero vamos a importar alguna de la librerías descritas en el primer notebook de este capíutlo.

Usaremos:

* `numpy` (vectores)
* `pandas`(DataFrames)
* `matplotlib` (visualización de bajo nivel)
* `seaborn` (visualización estadística). 

Ya tienen experiencia con estas cuatro bibliotecas, pero con este ejemplo profundizaremos un poco más en las dos últimas.

## `matplotlib`

`matplotlib` es una biblioteca de **bajo nivel** para visualización en Python, con un paradigma **imperativo**.

  * De bajo nivel quiere decir que entrega las primitivas gráficas necesarias para crear visualizaciones más complejtas.
  * Imperativo quiere decir que se focaliza en las instrucciones que recibe la biblioteca, ya que no _abstrae_ las operaciones o codificaciones visuales de modo que operemos sobre éstas.

## `seaborn`

`seaborn` es una biblioteca que se construye _sobre_ `matplotlib`, y que incluye gráficos estadísticos y algunas funcionalidades avanzadas de modelamiento y reducción de complejidad. Esto permite _hacer más con menos código_. 

Al mismo tiempo, `seaborn` incluye opciones para trabajar con la codificación visual de manera más efectiva que `matplotlib`, por ejemplo, a través de la facilitación de elección de paletas de colores, o de la detección de parámetros siguiendo buenas prácticas.

Y para completar más esta oferta, `seaborn` es directamente compatible con `pandas`. `matplotlib` también, pero no de manera nativa, por lo que todo lo que se relaciona con `pandas` requiere más trabajo.

En el ejemplo de abajo vemos que no solamente hizo el gráfico, sino que incluso etiquetó automáticamente el eje $x$.

Sin embargo, en `seaborn` la idea no es reemplazar los métodos de `matplotlib`. En realidad, lo que haremos será trabajar de manera diferente. Lo primero que debemos hacer es cambiar el formato de los datos. 

En `seaborn` se trabaja con **tidy data**. O datos ordenaditos.

(Aquí hay dos explicaciones interesantes sobre _tidy data_: en [Python](http://www.jeannicholashould.com/tidy-data-in-python.html). El paper que explica el razonamiento detrás está [aquí](http://courses.had.co.nz.s3-website-us-east-1.amazonaws.com/12-rice-bdsi/slides/07-tidy-data.pdf)).

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Esto configura la apariencia de los gráficos utilizando configuraciones de seaborn
sns.set(context='poster', style='ticks', palette='inferno')

# Esto es una instrucción de Jupyter que hace que los gráficos se desplieguen en el notebook
%matplotlib inline

# la encuesta tiene tantas columnas que hacemos esto para imprimirlas todas
pd.set_option('max_columns', 1000)

## Los datos: endimiento y limpieza

Esto involucra:

  * Inspeccionar el contenido de la base de datos para saber qué cargar.
  * Ajustar las perillas de los métodos para cargar datos, de modo que lean lo que queremos que leamos, y no se equivoquen.
  * Realizar visualización exploratoria para saber si está todo en orden.
  * Eliminar observaciones que no sirven, sea porque vienen _sucias_, _incompletas_, o bien no son relevantes para nuestro análisis.
  
  
El siguiente comando muestra los parámetros de la EOD.

In [ ]:
!ls ./carto-en-python/input/2012_eod_stgo/EOD_STGO/Tablas_parametros/

La siguiente consulta muestra las tablas de información de la EOD.

In [ ]:
!ls ./carto-en-python/input/2012_eod_stgo/EOD_STGO/

Si queremos visualizar, por ejemplo información de viajes:

In [ ]:
trip_data = pd.read_csv('./carto-en-python/input/2012_eod_stgo/EOD_STGO/viajes.csv', sep=';')
trip_data.head()

In [ ]:
trip_data.columns

Por ejemplo, notemos que las columnas ComunaOrigen y ComunaDestino tienen valores numéricos que deben ser IDs.

In [ ]:
municipalities = pd.read_csv('./carto-en-python/input/2012_eod_stgo/EOD_STGO/Tablas_parametros/Comuna.csv', sep=',')
municipalities.head()

Entonces, seguramente las comunas de origen/destino de un viaje contienen esos valores, pero codificados.

In [ ]:
trip_data.merge(municipalities, left_on='ComunaOrigen', right_on='Id').iloc[0]

In [ ]:
def decode_column(df, fname, col_name, index_col='Id', target_col=None, value_col=None, sep=';', encoding='utf-8'):
    '''
    param :df: DataFrame del que leeremos una columna.
    param :fname: nombre del archivo que contiene los valores a decodificar.
    param :col_name: nombre de la columna que queremos decodificar.
    param :index_col: nombre de la columna en el archivo @fname que tiene los índices que codifican @col_name
    param :value_col: nombre de la columna en el archivo @fname que tiene los valores decodificados
    param :sep: carácter que separa los valores en @fname. 
    param :encoding: identificación del _character set_ que utiliza el archivo. Usualmente es utf-8, si no funciona,
                     se puede probar con iso-8859-1.
    '''
    if value_col is None:
        value_col = 'value'
        
    if target_col is None:
        target_col = col_name
        
    values_df = pd.read_csv(fname, sep=sep, index_col=index_col, names=[index_col, value_col], header=0,
                            dtype={index_col: np.float64}, encoding=encoding)
    
    src_df = df.loc[:,(col_name,)]
    
    return pd.Series(src_df.join(values_df, on=col_name)[value_col], name=target_col)

decode_column(trip_data.head(10), './carto-en-python/input/2012_eod_stgo/EOD_STGO/Tablas_parametros/Comuna.csv', 'ComunaOrigen', sep=',')

Estudiaremos el uso de modos de viaje. Hay una tabla de parámetros que tiene esa información:

In [ ]:
!head ./carto-en-python/input/2012_eod_stgo/EOD_STGO/Tablas_parametros/Modo.csv

In [ ]:
trip_data.MediosUsados.value_counts()

Existe otra tabla llamada ViajesDifusion:

In [ ]:
trip_mode = pd.read_csv('./carto-en-python/input/2012_eod_stgo/EOD_STGO/ViajesDifusion.csv', sep=';', index_col='Viaje')
trip_mode.head()

In [ ]:
decode_column(trip_mode.sample(15), './carto-en-python/input/2012_eod_stgo/EOD_STGO/Tablas_parametros/ModoDifusion.csv', 'ModoDifusion', sep=';', index_col='ID', encoding='iso-8859-1')

Al parecer es más adecuada para lo que queremos.

In [ ]:
trip_data.head().join(trip_mode)

Hagamos fast-forward :)

In [ ]:
trip_data = (pd.read_csv('./carto-en-python/input/2012_eod_stgo/EOD_STGO/viajes.csv', sep=';', index_col='Viaje')
             .assign(ComunaOrigen=lambda x: decode_column(x, './carto-en-python/input/2012_eod_stgo/EOD_STGO/Tablas_parametros/Comuna.csv', 'ComunaOrigen', sep=','))
             .assign(ComunaDestino=lambda x: decode_column(x, './carto-en-python/input/2012_eod_stgo/EOD_STGO/Tablas_parametros/Comuna.csv', 'ComunaDestino', sep=','))
             .assign(SectorOrigen=lambda x: decode_column(x, './carto-en-python/input/2012_eod_stgo/EOD_STGO/Tablas_parametros/Sector.csv', 'SectorOrigen', sep=';'))
             .assign(SectorDestino=lambda x: decode_column(x, './carto-en-python/input/2012_eod_stgo/EOD_STGO/Tablas_parametros/Sector.csv', 'SectorDestino', sep=';'))
             .join(pd.read_csv('./carto-en-python/input/2012_eod_stgo/EOD_STGO/ViajesDifusion.csv', sep=';', index_col='Viaje'))
             .assign(ModoDifusion=lambda x: decode_column(x, './carto-en-python/input/2012_eod_stgo/EOD_STGO/Tablas_parametros/ModoDifusion.csv', 'ModoDifusion', sep=';', index_col='ID', encoding='iso-8859-1'))
             .assign(Proposito=lambda x: decode_column(x, './carto-en-python/input/2012_eod_stgo/EOD_STGO/Tablas_parametros/Proposito.csv', 'Proposito', sep=';'))
             .join(pd.read_csv('./carto-en-python/input/2012_eod_stgo/EOD_STGO/DistanciaViaje.csv', sep=';', index_col='Viaje'))
            )

trip_data.sample(10)

Todavía hay un par de asuntos que queremos resolver. Por ejemplo, los tiempos de viaje y las coordenadas geográficas. Antes de eso, limpiemos el dataset sacando valores nulos e inválidos.

In [ ]:
trip_data = trip_data.dropna(subset=['HoraIni', 'HoraFin', 'SectorOrigen', 'SectorDestino', 'ComunaOrigen', 'ComunaDestino', 'Proposito', 'ModoDifusion'])

In [ ]:
trip_data['HoraIni'] = trip_data.HoraIni.str.split(':').map(lambda x: pd.Timedelta(hours=int(x[0]), minutes=int(x[1])))
trip_data['HoraIni'].describe()

In [ ]:
trip_data['HoraFin'] = trip_data.HoraFin.str.split(':').map(lambda x: pd.Timedelta(hours=int(x[0]), minutes=int(x[1])))
trip_data['HoraFin'].describe()

En algunos casos, la hora de fin es _menor_ a la hora de inicio. Eso lo podemos corregir:

In [ ]:
trip_data['HoraFin'][trip_data['HoraFin'] < trip_data['HoraIni']] = trip_data['HoraFin'][trip_data['HoraFin'] < trip_data['HoraIni']] + pd.Timedelta(days=1)

In [ ]:
trip_data.sample(15)

¡Ya podemos hacer cosas!

In [ ]:
sns.distplot(trip_data.HoraIni / pd.Timedelta(hours=1))

In [ ]:

trip_data['departure_time'] = trip_data.HoraIni / pd.Timedelta(hours=1)

In [ ]:
sns.distplot(trip_data.HoraIni / pd.Timedelta(hours=1))

In [ ]:
asdf = sns.FacetGrid(data=trip_data, col='Proposito', col_wrap=3, sharey=False, aspect=1.75, size=3)
asdf.map(sns.distplot, 'departure_time')
asdf.set(xlim=(0,24))

In [ ]:
people_data = (pd.read_csv('./carto-en-python/input/2012_eod_stgo/EOD_STGO/personas.csv', sep=';', index_col='Persona')
               .assign(Sexo=lambda x: decode_column(x, './carto-en-python/input/2012_eod_stgo/EOD_STGO/Tablas_parametros/Sexo.csv', 'Sexo', sep=';'))
               .assign(Relacion=lambda x: decode_column(x, './carto-en-python/input/2012_eod_stgo/EOD_STGO/Tablas_parametros/Relacion.csv', 'Relacion', sep=';'))
               .assign(Ocupacion=lambda x: decode_column(x, './carto-en-python/input/2012_eod_stgo/EOD_STGO/Tablas_parametros/Ocupacion.csv', 'Ocupacion', sep=';'))
              )
people_data.head()

In [ ]:
trip_data = trip_data.join(people_data.loc[:,('Sexo', 'Relacion', 'Ocupacion')], on='Persona')

In [ ]:
trip_data.sample(5)

Ahora podemos desagregar el gráfico por más variables:

In [ ]:
g = sns.FacetGrid(data=trip_data, col='Proposito', hue='Sexo', col_wrap=3, sharey=False, aspect=1.75, size=3, palette='PuOr')
g.map(sns.distplot, 'departure_time')
g.set(xlim=(0,24))
g.add_legend()

In [ ]:
g = sns.FacetGrid(data=trip_data, col='Proposito', hue='Sexo', col_wrap=3, sharey=False, aspect=1.75, size=3, palette='PuOr')
g.map(sns.distplot, 'departure_time')
g.set(xlim=(0,24))
g.add_legend()

In [ ]:
ocupations = trip_data.Ocupacion.value_counts()
ocupations

In [ ]:
g = sns.FacetGrid(data=trip_data, col='Proposito', hue='Ocupacion', col_wrap=3, sharey=False, aspect=1.75, size=3, palette='husl',
                 hue_order=ocupations.index[0:4])
g.map(sns.distplot, 'departure_time')
g.set(xlim=(0,24))
g.add_legend()

Continuará ...